In [1]:
import cherrypy
import json
import redis
import uuid
from redis.commands.json.path import Path

#PERSONAL PARAMETERS YOU CAN OBTAIN FROM YOUR REDIS ACCOUNT

#PERSONAL PARAMETERS YOU CAN OBTAIN FROM YOUR REDIS ACCOUNT
REDIS_HOST = "****"
REDIS_PORT = 00000
REDIS_USERNAME = "****"
REDIS_PASSWORD = "****"




# Connect to Redis server
redis_client = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, username=REDIS_USERNAME, password=REDIS_PASSWORD)
is_connected = redis_client.ping()
print('Redis Connected:', is_connected)


# -----------------------------------------------------------endpoint/status-----------------------------------------------------------
class serverStatus(object):
    exposed = True  # this makes the endpoint available for the users

    def GET(self, *path, **query):
        response_dict = {
            'status': 'online'
        }
        response = json.dumps(response_dict)

        return response

# --------------------------------------------------------------/devices-----------------------------------------------------------------
class Devices(object):
    exposed = True #this makes the endpoint available for the users
    def GET(self, *path, **query): #path and query are used in case you have them in the http request path is a list, query is a dict
        keys = redis_client.keys('macs:*')
        items = []

        for key in keys:
            key = key.decode()
            uid = key.removeprefix('macs:')
            items.append(uid)
            
        response_dict = {
            'macs' : items
        }
        
        cherrypy.response.status = "200 - OK: Everything worked as expected."
        response = json.dumps(response_dict)
        return response
    
    @cherrypy.tools.json_out()
    def DELETE(self):
        redis_client.flushall()
        return {'status': 'success', 'message': 'Redis database flushed successfully'}

# ---------------------------------------------------------/device/{mac_address}-----------------------------------------------------------------
class SingleDevice(object):
    '''Retrieves information of the device's specific timeseries with the specified MAC address
    in the specified time range.'''
    exposed = True
    def GET(self, *path, **query):
        ''' 
        o 200 – OK: Everything worked as expected.
        o 400 – Bad Request: missing MAC address.
        o 400 – Bad Request: missing start time.
        o 400 – Bad Request: missing end time.
        o 404 – Not Found: invalid MAC address.
        '''

        #------- Error handling start 
        # missing MAC address
        if(len(path)) != 1:
            raise cherrypy.HTTPError(400, 'Bad Request: missing MAC address.')  #compliant to the documentation
        
        # missing start time 
        query_from = query.get('from', None)
        if query_from is None:
            raise cherrypy.HTTPError(400, 'Bad Request: missing start time.')  #compliant to the documentation
                
        # missing end time
        query_to = query.get('to', None)
        if query_to is None:
            raise cherrypy.HTTPError(400, 'Bad Request: missing end time.')  #compliant to the documentation
        
        # invalid MAC address
        keys = redis_client.keys('macs:*')
        macs = []
        for key in keys:
            key = key.decode()
            uid = key.removeprefix('macs:')
            macs.append(uid)
        
        mac_address_from_path = path[0] #path contains something (checked before), now I check if it exists in Redis
        if mac_address_from_path not in macs:
            raise cherrypy.HTTPError(404, '404 – Not Found: invalid MAC address.') #compliant to the documentation
        #------- Error handling finish

        #Extracting for that mac address in that timeframe the values for all the emotions timeseries
        timeseries_Ang = redis_client.ts().range(f'{mac_address_from_path}:ANG',query_from, query_to)
        timeseries_Neu = redis_client.ts().range(f'{mac_address_from_path}:NEU',query_from, query_to) 
        timeseries_Hap = redis_client.ts().range(f'{mac_address_from_path}:HAP',query_from, query_to) 
        # add if new label
        
        # Building the response_dictionary
        response_dict = {
            'mac_address': mac_address_from_path,
            'labels' : ['ANG, NEU, HAP'], # add if new label
            'timeseries_ANG': timeseries_Ang,
            'timeseries_NEU': timeseries_Neu,
            'timeseries_HAP': timeseries_Hap
            }
        #If everything worked fine
        cherrypy.response.status = "200 - OK: Everything worked as expected."
        
        response = json.dumps(response_dict)
        return response

    
    def DELETE(self, *path, **query):
        '''
            o 200 – OK: Everything worked as expected.
            o 400 – Bad Request: missing MAC address.
            o 404 – Not Found: invalid MAC address.
        '''
        # missing MAC address
        if(len(path)) != 1:
            raise cherrypy.HTTPError(400, 'Bad Request: missing MAC address.')  #compliant to the documentation

        # invalid MAC address
        keys = redis_client.keys('macs:*')
        macs = []
        for key in keys:
            key = key.decode()
            uid = key.removeprefix('macs:')
            macs.append(uid)
        
        mac_address_from_path = path[0] #path contains something (checked before), now I check if it exists in Redis
        if mac_address_from_path not in macs:
            raise cherrypy.HTTPError(404, '404 – Not Found: invalid MAC address.') #compliant to the documentation
        
        #Deleting the timeseries
        redis_client.delete(f'{mac_address_from_path}:ANG')
        redis_client.delete(f'{mac_address_from_path}:NEU')
        redis_client.delete(f'{mac_address_from_path}:HAP')

        redis_client.delete(f'macs:{mac_address_from_path}')

        #If everything worked fine
        cherrypy.response.status = "200 - OK: Everything worked as expected."
        return

   


if __name__ == '__main__':
    conf = {
        '/': {'request.dispatch': cherrypy.dispatch.MethodDispatcher()}
        }

    #It's important to map each class to the correct endpoint later
    cherrypy.tree.mount(serverStatus(), '/serverstatus', conf)
    cherrypy.tree.mount(Devices(), '/devices', conf)
    cherrypy.tree.mount(SingleDevice(), '/device', conf)
        
    cherrypy.config.update({'server.socket_host': '0.0.0.0'})
    cherrypy.config.update({'server.socket_port': 8080})
    cherrypy.engine.start()
    cherrypy.engine.block()
   

[21/Mar/2023:16:26:24] ENGINE Bus STARTING
[21/Mar/2023:16:26:24] ENGINE Started monitor thread 'Autoreloader'.
[21/Mar/2023:16:26:24] ENGINE Serving on http://0.0.0.0:8080
[21/Mar/2023:16:26:24] ENGINE Bus STARTED
Redis Connected: True


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f648dc53-4e62-47c1-9023-194409755340' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>